# **Import Statements**

In [1]:
import sys
#root = "C:\\Users\\saman\\OneDrive\\Documents\\GitHub\\Eyelid-Modeling\\" #laptop path
root = "C:\\Users\\Samantha\\Documents\\GitHub\\Eyelid-Modeling\\" #desktop path
sys.path.append(root)

from utils.experiment_manager import Experiment
from models.EMG_RNN import EMG_RNN_Wrapper
from utils.data_management import Mat2TVT

import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle

%load_ext autoreload
%autoreload 2

torch.cuda.is_available()

True

# **Data Reorganization** <br>


In [2]:
eye_bool = np.asarray([True, True, False, True, True, False, False, False])
electrode_list = ['u1', 'u2', 'u3', 'u4', 't2']
marker_list = ['u1', 'u2', 'u3', 'u4', 'u5']
filepath = root + 'eyeliddata.mat'

train = 0.7
val = 0.2
test = 0.1

reorganizer = Mat2TVT(eye_bool, electrode_list, marker_list, filepath)
_, _, identifier = reorganizer.load_data()
X, y = reorganizer.DMVC_norm()
TVT_dict = reorganizer.TVT_split(train, val, test)

# **Run Experiments**

In [3]:
params = {
    't_win': np.arange(10,51,1, dtype = float),
    't_stride': np.arange(10, 51, 1, dtype = float),
    't_lookahead': np.arange(50, 101, 1, dtype = float),
    'win_mode': ['win_fft', 'win_rect'],
    'p_transform': np.linspace(0, 1, 100, dtype = float),
    'sigma': np.linspace(0, 5, 100, dtype = float),
    'p_mask': np.linspace(0, 0.5, 100, dtype = float),
    'batch_size': [4, 8, 16, 32, 64, 128],
    'learning_rate': np.linspace(1e-3, 1e-8, 500, dtype = float),
    'dropout': np.linspace(0, 1, 100, dtype = float),
    'RNN_hdim': [4, 8, 16, 32, 64, 128],
    'RNN_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
}

experiment = Experiment()
model = EMG_RNN_Wrapper()
best_params = experiment(params, TVT_dict, model, n_trials = 250, epochs = 100, patience = 25, gcollect = False) 

print("Best Params:\n_________________________________________________________________________________________________________")
for key, value in best_params.items():
    print('    {}: {}'.format(key, value))

[I 2025-03-14 17:39:30,200] A new study created in memory with name: no-name-f71e6fd5-b68b-435d-b33a-c23045dc917b


Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------
Epoch 1
-------------------------------
Validation Error: 
 Max Marker Error: 3181.3%, Avg Marker Error: 115.9%, Avg loss: 4292.465332 

Epoch 2
-------------------------------
Validation Error: 
 Max Marker Error: 3843.3%, Avg Marker Error: 107.5%, Avg loss: 4195.825684 

Epoch 3
-------------------------------
Validation Error: 
 Max Marker Error: 1853.1%, Avg Marker Error: 113.4%, Avg loss: 4100.846191 

Epoch 2
-------------------------------
Validation Error: 
 Max Marker Error: 2307.2%, Avg Marker Error: 100.2%, Avg loss: 4102.571289 

Epoch 4
-------------------------------
Validation Error: 
 Max Marker Error: 6902.5%, Avg Marker Error: 110.9%, Avg loss: 4297.227539 

Epoch 2
---------------------------

[I 2025-03-14 17:43:39,529] Trial 3 finished with value: 2892.867431640625 and parameters: {'t_win': np.float64(50.0), 't_stride': np.float64(44.0), 't_lookahead': np.float64(80.0), 'win_mode': 'win_fft', 'p_transform': np.float64(0.24242424242424243), 'sigma': np.float64(2.2222222222222223), 'p_mask': np.float64(0.4545454545454546), 'batch_size': 128, 'learning_rate': np.float64(0.0009619242284569138), 'dropout': np.float64(0.5454545454545455), 'RNN_hdim': 32, 'RNN_depth': 3}. Best is trial 3 with value: 2892.867431640625.


Validation Error: 
 Max Marker Error: 6386.9%, Avg Marker Error: 92.3%, Avg loss: 2907.419434 

Done!
Validation Error: 
 Max Marker Error: 1999.4%, Avg Marker Error: 97.2%, Avg loss: 4096.664062 

Epoch 28
-------------------------------
Epoch 1
-------------------------------
Validation Error: 
 Max Marker Error: 1040.2%, Avg Marker Error: 94.6%, Avg loss: 4312.195801 

Epoch 3
-------------------------------
Validation Error: 
 Max Marker Error: 1403.0%, Avg Marker Error: 98.1%, Avg loss: 4033.217529 

Epoch 35
-------------------------------
Validation Error: 
 Max Marker Error: 1652.6%, Avg Marker Error: 105.9%, Avg loss: 4228.266602 

Epoch 28
-------------------------------
Validation Error: 
 Max Marker Error: 3350.5%, Avg Marker Error: 86.2%, Avg loss: 2984.194092 

Epoch 5
-------------------------------
Validation Error: 
 Max Marker Error: 1980.5%, Avg Marker Error: 97.0%, Avg loss: 4099.393555 

Epoch 29
-------------------------------
Validation Error: 
 Max Marker Error:

[I 2025-03-14 17:46:49,956] Trial 6 finished with value: 4025.963623046875 and parameters: {'t_win': np.float64(31.0), 't_stride': np.float64(29.0), 't_lookahead': np.float64(79.0), 'win_mode': 'win_rect', 'p_transform': np.float64(0.17171717171717174), 'sigma': np.float64(0.45454545454545453), 'p_mask': np.float64(0.015151515151515152), 'batch_size': 64, 'learning_rate': np.float64(0.0002444965330661322), 'dropout': np.float64(0.5757575757575758), 'RNN_hdim': 16, 'RNN_depth': 10}. Best is trial 3 with value: 2892.867431640625.


...

Validation Error: 
 Max Marker Error: 5070.1%, Avg Marker Error: 92.0%, Avg loss: 1964.256836 

Stopped early after epoch: 66
Done!
Validation Error: 
 Max Marker Error: 5927.3%, Avg Marker Error: 114.3%, Avg loss: 2396.583496 

Epoch 45
-------------------------------
Validation Error: 
 Max Marker Error: 3837.9%, Avg Marker Error: 86.7%, Avg loss: 1836.854492 

Epoch 43
-------------------------------
Validation Error: 
 Max Marker Error: 4457.8%, Avg Marker Error: 95.5%, Avg loss: 1618.344604 

Epoch 46
-------------------------------
Validation Error: 
 Max Marker Error: 4504.2%, Avg Marker Error: 85.8%, Avg loss: 1750.584351 

Epoch 44
-------------------------------
Validation Error: 
 Max Marker Error: 3850.4%, Avg Marker Error: 97.5%, Avg loss: 1683.950562 

Epoch 47
-------------------------------
Validation Error: 
 Max Marker Error: 3453.4%, Avg Marker Error: 107.8%, Avg loss: 1866.744751 

Epoch 48
-------------------------------
Validation Error: 
 Max Marker Error: 5307.6

[I 2025-03-15 06:57:22,655] Trial 246 finished with value: 1552.05419921875 and parameters: {'t_win': np.float64(49.0), 't_stride': np.float64(12.0), 't_lookahead': np.float64(68.0), 'win_mode': 'win_fft', 'p_transform': np.float64(0.5050505050505051), 'sigma': np.float64(1.1616161616161615), 'p_mask': np.float64(0.3484848484848485), 'batch_size': 4, 'learning_rate': np.float64(0.000749501503006012), 'dropout': np.float64(0.07070707070707072), 'RNN_hdim': 128, 'RNN_depth': 2}. Best is trial 197 with value: 1436.719970703125.


Validation Error: 
 Max Marker Error: 3505.0%, Avg Marker Error: 98.2%, Avg loss: 1661.617798 

Stopped early after epoch: 58
Done!
Validation Error: 
 Max Marker Error: 12216.6%, Avg Marker Error: 106.4%, Avg loss: 2148.304443 

Epoch 53
-------------------------------
Validation Error: 
 Max Marker Error: 4778.2%, Avg Marker Error: 89.5%, Avg loss: 1730.798462 

Epoch 54
-------------------------------
Validation Error: 
 Max Marker Error: 5468.1%, Avg Marker Error: 94.2%, Avg loss: 1871.444092 

Epoch 55
-------------------------------
Validation Error: 
 Max Marker Error: 4841.6%, Avg Marker Error: 94.0%, Avg loss: 1839.031616 

Epoch 56
-------------------------------
Validation Error: 
 Max Marker Error: 4792.1%, Avg Marker Error: 91.8%, Avg loss: 1796.160522 

Epoch 57
-------------------------------
Validation Error: 
 Max Marker Error: 5760.2%, Avg Marker Error: 86.2%, Avg loss: 1754.171631 

Epoch 58
-------------------------------
Validation Error: 
 Max Marker Error: 7446.9

[I 2025-03-15 07:24:56,640] Trial 243 finished with value: 1649.830810546875 and parameters: {'t_win': np.float64(49.0), 't_stride': np.float64(12.0), 't_lookahead': np.float64(51.0), 'win_mode': 'win_fft', 'p_transform': np.float64(0.9090909090909092), 'sigma': np.float64(2.2727272727272725), 'p_mask': np.float64(0.42424242424242425), 'batch_size': 4, 'learning_rate': np.float64(0.0004949950300601202), 'dropout': np.float64(0.30303030303030304), 'RNN_hdim': 64, 'RNN_depth': 2}. Best is trial 197 with value: 1436.719970703125.


Validation Error: 
 Max Marker Error: nan%, Avg Marker Error: nan%, Avg loss:      nan 

Done!
Best Params:
_________________________________________________________________________________________________________
    t_win: 49.0
    t_stride: 12.0
    t_lookahead: 51.0
    win_mode: win_fft
    p_transform: 0.7575757575757577
    sigma: 2.727272727272727
    p_mask: 0.2878787878787879
    batch_size: 16
    learning_rate: 0.0007174376953907815
    dropout: 0.6666666666666667
    RNN_hdim: 128
    RNN_depth: 2


# **Training Model with Best Params**

In [ ]:
metrics = experiment.run_experiment(best_params, TVT_dict, model, epochs = 1000, patience = 25)

'''
Saving Model, Training Metrics, and Best Params
'''
with open("training_metrics.pickle", "wb") as file:
    pickle.dump(metrics, file)
with open("best_params.pickle", "wb") as file:
    pickle.dump(best_params, file)

Epoch 1
-------------------------------
Validation Error: 
 Max Marker Error: 2442.6%, Avg Marker Error: 97.0%, Avg loss: 3484.519287 

Epoch 2
-------------------------------
Validation Error: 
 Max Marker Error: 1950.2%, Avg Marker Error: 91.1%, Avg loss: 3321.938477 

Epoch 3
-------------------------------
Validation Error: 
 Max Marker Error: 2464.7%, Avg Marker Error: 96.6%, Avg loss: 3295.598877 

Epoch 4
-------------------------------
Validation Error: 
 Max Marker Error: 2767.1%, Avg Marker Error: 89.7%, Avg loss: 3204.055908 

Epoch 5
-------------------------------
Validation Error: 
 Max Marker Error: 2952.2%, Avg Marker Error: 96.6%, Avg loss: 3194.789307 

Epoch 6
-------------------------------
Validation Error: 
 Max Marker Error: 2742.7%, Avg Marker Error: 95.7%, Avg loss: 3220.261230 

Epoch 7
-------------------------------
Validation Error: 
 Max Marker Error: 2280.6%, Avg Marker Error: 92.2%, Avg loss: 3105.490723 

Epoch 8
-------------------------------
Validati

: 

# **Plotting Loss Metrics**

In [ ]:
plt.plot(metrics['Training Loss'], label = 'Training Loss', marker = '.')
plt.plot(metrics['Validation Loss'], label = 'Validation Loss', marker = '.')
plt.title('Loss vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(metrics['Training Avg Marker Error'], label = 'Training Error', marker = '.')
plt.plot(metrics['Validation Avg Marker Error'], label = 'Validation Error', marker = '.')
plt.title('Average Marker Error vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Average Marker Error')
plt.legend()
plt.show()

plt.plot(metrics['Training Max Marker Error'], label = 'Training Error', marker = '.')
plt.plot(metrics['Validation Max Marker Error'], label = 'Validation Error', marker = '.')
plt.title('Max Marker Error vs. Epoch')
plt.xlabel('Epochs')
plt.ylabel('Max Marker Error')
plt.legend()
plt.show()